In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import dask.dataframe as dd
import dask.array as da
from dask.distributed import Client
from dask_ml.metrics import accuracy_score
from dask_ml.model_selection import train_test_split
from tqdm import tqdm

In [2]:
# Main class for layer
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    def __call__(self, x):
        return self.forward(x)

    def forward(self, input):
        raise NotImplementedError

    def backward(self, up_gradient):
        raise NotImplementedError

    def step(self, learning_rate):
        pass


In [3]:
# Loss function class
class Loss:
    def __init__(self):
        self.prediction = None
        self.target = None
        self.loss = None

    def __call__(self, prediction, target):
        return self.forward(prediction, target)

    def forward(self, prediction, target):
        raise NotImplementedError

    def backward(self):
        raise NotImplementedError

In [4]:
# Multi-Layer Perceptron (MLP)
class MLP:
    def __init__(self, layers, loss, learning_rate):
        self.layers = layers
        self.loss_fn = loss
        self.learning_rate = learning_rate

    def __call__(self, input):
        return self.forward(input)

    def forward(self, input):
        for layer in self.layers:
            input = layer(input)
        return input

    def backward(self):
        up_gradient = self.loss_fn.backward()
        for layer in reversed(self.layers):
            up_gradient = layer.backward(up_gradient)

    def loss(self, prediction, target):
        return self.loss_fn(prediction, target)

    def update(self):
        for layer in self.layers:
            layer.step(self.learning_rate)

    def train(self, x_train, y_train, epochs, batch_size):
        losses = []
        for epoch in tqdm(range(epochs), desc="Training Progress"):
            running_loss = 0
            for i in range(0, x_train.shape[0], batch_size):
                x_batch = x_train[i : i + batch_size]
                y_batch = y_train[i : i + batch_size]

                prediction = self.forward(x_batch)

                # Compute Loss and ensure computation
                loss = self.loss_fn(prediction, y_batch).mean().compute()
                running_loss += loss

                self.backward()
                self.update()

            running_loss /= (x_train.shape[0] / batch_size)
            losses.append(running_loss)
            print(f"Epoch {epoch+1}/{epochs} | Loss: {running_loss:.4f}")

        return losses

In [5]:
# Linear layer
class Linear(Layer):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.w = 0.1 * da.random.normal(size=(input_size, output_size), chunks=(input_size, output_size))
        self.b = da.zeros((1, output_size), chunks=(1, output_size))
        self.dw = da.zeros_like(self.w)
        self.db = da.zeros_like(self.b)

    def forward(self, input):
        self.input = input
        return da.dot(self.input, self.w) + self.b

    def backward(self, up_gradient):
        self.dw = da.dot(self.input.T, up_gradient)
        self.db = da.sum(up_gradient, axis=0, keepdims=True)
        return da.dot(up_gradient, self.w.T)

    def step(self, learning_rate):
        self.w -= learning_rate * self.dw
        self.b -= learning_rate * self.db

In [6]:
# ReLU activation function
class ReLU(Layer):
    def forward(self, input):
        self.input = input
        return da.maximum(0, input)

    def backward(self, up_gradient):
        return up_gradient * (self.input > 0)

In [7]:

# CrossEntropy loss
class CrossEntropy(Loss):
    def forward(self, prediction, target):
        exp_pred = da.exp(prediction - da.max(prediction, axis=1, keepdims=True))
        self.softmax = exp_pred / da.sum(exp_pred, axis=1, keepdims=True)
        self.target = target
        return -da.mean(da.sum(target * da.log(self.softmax + 1e-15), axis=1))

    def backward(self):
        return (self.softmax - self.target) / self.target.shape[0]

In [8]:
# Mean Squared Error loss
class MSE(Loss):
    def forward(self, prediction, target):
        self.prediction = prediction
        self.target = target
        return da.mean((prediction - target) ** 2)

    def backward(self):
        return 2 * (self.prediction - self.target) / len(self.target)

In [9]:
print(pd.__version__)  

2.2.3


In [10]:
# Simulate data using Dask directly for faster loading
def simulate_data():
    client = Client()  # Start Dask client

    n_samples = 500000
    n_features = 100

    # Generate data directly as Dask arrays
    X = da.random.random((n_samples, n_features), chunks=(100_000, n_features))
    y = da.random.randint(0, 2, size=(n_samples,), chunks=(100_000,))

    # Convert to Dask DataFrame
    X_dd = dd.from_dask_array(X, columns=[f'feature{i}' for i in range(n_features)])
    y_dd = dd.from_dask_array(y, columns=['target'])

    # Split into training & test sets (85% train, 15% test)
    X_train, X_test, y_train, y_test = train_test_split(X_dd, y_dd, test_size=0.15, random_state=42)

    # Convert back to Dask arrays
    return X_train.to_dask_array(lengths=True), X_test.to_dask_array(lengths=True), \
           y_train.to_dask_array(lengths=True), y_test.to_dask_array(lengths=True)

In [11]:
# Create network layers (2-layer neural network)
layers = [
    Linear(100, 50),
    ReLU(),
    Linear(50, 2)
]

# Load data
x_train, x_test, y_train, y_test = simulate_data()

# Create the model
model = MLP(layers, CrossEntropy(), learning_rate=0.01)

# Train the model
losses = model.train(x_train=x_train, y_train=y_train, epochs=30, batch_size=1000)

# Compute accuracy
y_pred = model(x_test).compute()
accuracy = accuracy_score(y_test.compute(), np.argmax(y_pred, axis=1))
print(f"Accuracy: {accuracy}")

# Plot losses
plt.plot(losses)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training Loss Curve")
plt.show()


/opt/homebrew/Caskroom/miniconda/base/envs/datascience/lib/python3.12/site-packages/dask_ml/model_selection/_split.py:464: FutureWarning: The default value for 'shuffle' must be specified when splitting DataFrames. In the future DataFrames will automatically be shuffled within blocks prior to splitting. Specify 'shuffle=True' to adopt the future behavior now, or 'shuffle=False' to retain the previous behavior.
  warnings.warn(
Training Progress:   0%|          | 0/30 [40:12<?, ?it/s]Task exception was never retrieved
future: <Task finished name='Task-1239794' coro=<Client._gather.<locals>.wait() done, defined at /opt/homebrew/Caskroom/miniconda/base/envs/datascience/lib/python3.12/site-packages/distributed/client.py:2394> exception=AllExit()>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/datascience/lib/python3.12/site-packages/distributed/client.py", line 2403, in wait
    raise AllExit()
distributed.client.AllExit

2025-02-26 23:27:19,964 - dis

KeyboardInterrupt: 

2025-02-26 23:27:21,950 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/datascience/lib/python3.12/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/envs/datascience/lib/python3.12/asyncio/base_events.py", line 685, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/envs/datascience/lib/python3.12/site-packages/distributed/nanny.py", line 985, in run
    await worker.finished()
  File "/opt/homebrew/Caskroom/miniconda/base/envs/datascience/lib/python3.12/site-packages/distributed/core.py", line 494, in finished
    await self._event_finished.wait()
  File "/opt/homebrew/Caskroom/miniconda/base/envs/datascience/lib/python3.12/asyncio/locks.py", line 2